# NO PROFIT/LOSS TARGET
# LOG(X1) - LOG(X0)
# SHOULD CLOSE AT THE END OF THE DAY

In [ ]:
import sys
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import *
from sklearn.mixture import GaussianMixture
from sktime.forecasting.model_selection import SlidingWindowSplitter
from numba import jit
import joblib
import os
import shutil
import json
from sktime.forecasting.model_selection import SlidingWindowSplitter

sys.path.append(
    os.path.abspath(
        "/projects/genomic-ml/da2343/ml_project_2/unsupervised/kmeans"
    )
)
from utils import *

warnings.filterwarnings("ignore")

# Constants
INITIAL_CAPITAL = 100
RISK_FREE_RATE = 0.01

# Load trading parameters from CSV
trading_params = pd.read_csv("params.csv")
param_row = 0
param_dict = dict(
    trading_params.iloc[param_row, :]
)  # Assume first row of trading_params.csv

# Extract trading parameters
INSTRUMENT = param_dict["instrument"]
MAX_CLUSTER_LABELS = int(param_dict["max_cluster_labels"])
PRICE_HISTORY_LENGTH = int(param_dict["price_history_length"])
NUM_PERCEPTUALLY_IMPORTANT_POINTS = int(param_dict["num_perceptually_important_points"])
DISTANCE_MEASURE = int(param_dict["distance_measure"])
NUM_CLUSTERS = int(param_dict["num_clusters"])
ATR_MULTIPLIER = int(param_dict["atr_multiplier"])
CLUSTERING_ALGORITHM = param_dict["clustering_algorithm"]
RANDOM_SEED = int(param_dict["random_seed"])
TRAIN_PERIOD = int(param_dict["train_period"])
TEST_PERIOD = int(param_dict["test_period"])


# Define clustering algorithms
clustering_estimator_dict = {
    "kmeans": KMeans(n_clusters=NUM_CLUSTERS, random_state=RANDOM_SEED),
    "gaussian_mixture": GaussianMixture(
        n_components=NUM_CLUSTERS, covariance_type="tied", random_state=RANDOM_SEED
    ),
}



@jit(nopython=True)
def find_perceptually_important_points(price_data, num_points):
    point_indices = np.zeros(num_points, dtype=np.int64)
    point_prices = np.zeros(num_points, dtype=np.float64)
    point_indices[0], point_indices[1] = 0, len(price_data) - 1
    point_prices[0], point_prices[1] = price_data[0], price_data[-1]

    for current_point in range(2, num_points):
        max_distance, max_distance_index, insert_index = 0.0, -1, -1
        for i in range(1, len(price_data) - 1):
            left_adj = (
                np.searchsorted(point_indices[:current_point], i, side="right") - 1
            )
            right_adj = left_adj + 1
            distance = calculate_point_distance(
                price_data,
                point_indices[:current_point],
                point_prices[:current_point],
                i,
                left_adj,
                right_adj,
            )
            if distance > max_distance:
                max_distance, max_distance_index, insert_index = distance, i, right_adj

        point_indices[insert_index + 1 : current_point + 1] = point_indices[
            insert_index:current_point
        ]
        point_prices[insert_index + 1 : current_point + 1] = point_prices[
            insert_index:current_point
        ]
        point_indices[insert_index], point_prices[insert_index] = (
            max_distance_index,
            price_data[max_distance_index],
        )

    return point_indices, point_prices

@jit(nopython=True)
def calculate_point_distance(
    data, point_indices, point_prices, index, left_adj, right_adj
):
    time_diff = point_indices[right_adj] - point_indices[left_adj]
    price_diff = point_prices[right_adj] - point_prices[left_adj]
    slope = price_diff / time_diff
    intercept = point_prices[left_adj] - point_indices[left_adj] * slope
    x, y = index, data[index]

    if DISTANCE_MEASURE == 1:
        return (
            (point_indices[left_adj] - x) ** 2 + (point_prices[left_adj] - y) ** 2
        ) ** 0.5 + (
            (point_indices[right_adj] - x) ** 2 + (point_prices[right_adj] - y) ** 2
        ) ** 0.5
    elif DISTANCE_MEASURE == 2:
        return abs((slope * x + intercept) - y) / (slope**2 + 1) ** 0.5
    else:  # DISTANCE_MEASURE == 3
        return abs((slope * x + intercept) - y)

def prepare_data(price_subset, instrument_dict):
    data_list = []
    scaler = StandardScaler()
    
    # Get instrument's high liquidity hours
    liquidity_start = instrument_dict['market_hours']['high_liquidity_start']
    liquidity_end = instrument_dict['market_hours']['high_liquidity_end']

    for index in range(PRICE_HISTORY_LENGTH, len(price_subset)):
        # Get price history for PIP calculation
        price_history = (
            price_subset["close"]
            .iloc[max(0, index - PRICE_HISTORY_LENGTH) : index]
            .values
        )
        if len(price_history) < PRICE_HISTORY_LENGTH:
            break

        # Current row index
        j = index - 1
        current_time = price_subset.index[j]
        current_hour = current_time.hour
        
        # Check if current time is within high liquidity period
        # Handle cases where high liquidity period crosses midnight
        is_liquid_time = False
        if liquidity_start <= liquidity_end:
            is_liquid_time = liquidity_start <= current_hour < liquidity_end
        else:  # Period crosses midnight
            is_liquid_time = current_hour >= liquidity_start or current_hour < liquidity_end
            
        if not is_liquid_time:
            continue
            
        # Find next day's 00:00
        next_day = current_time.date() + pd.Timedelta(days=1)
        target_time = pd.Timestamp.combine(
            next_day, 
            pd.Timestamp('00:00').time()
        )
        
        # Get the EOD price (00:00 next day)
        eod_data = price_subset[price_subset.index <= target_time]
        if len(eod_data) == 0:
            continue
            
        eod_row = eod_data.iloc[-1]
        
        # Calculate PIPs and scale them
        _, important_points = find_perceptually_important_points(
            price_history, NUM_PERCEPTUALLY_IMPORTANT_POINTS
        )
        scaled_points = scaler.fit_transform(important_points.reshape(-1, 1)).flatten()
        
        # Create data point with scaled PIPs
        data_point = {
            f"price_point_{i}": scaled_points[i]
            for i in range(NUM_PERCEPTUALLY_IMPORTANT_POINTS)
        }

        # Add time features
        data_point.update(
            price_subset.iloc[j][
                ["year", "month", "day_of_week", "hour", "minute"]
            ].to_dict()
        )
        
        # Calculate log return from current point to EOD (next day 00:00)
        data_point["trade_outcome"] = (eod_row["log_close"] -  price_subset["log_close"].iloc[j])
        # data_point["trade_actual_outcome"] = (eod_row["close"] -  price_subset["close"].iloc[j])
        data_list.append(data_point)

    return pd.DataFrame(data_list)

def evaluate_cluster_performance_df(price_data_df, train_best_clusters_df, clustering_model):
    # Prepare features for prediction
    price_point_columns = [f"price_point_{i}" for i in range(NUM_PERCEPTUALLY_IMPORTANT_POINTS)]
    feature_columns = price_point_columns + ["day_of_week", "hour", "minute"]
    
    # Predict clusters for test data
    price_features = price_data_df[feature_columns].values
    price_data_df["cluster_label"] = clustering_model.predict(price_features)
    
    # Get the best cluster label and its direction from training
    best_cluster = train_best_clusters_df.iloc[0]  # Since we only have one best cluster
    cluster_label = best_cluster['cluster_label']
    trade_direction = best_cluster['trade_direction']
    
    # Get trades for the best cluster
    cluster_data = price_data_df[price_data_df['cluster_label'] == cluster_label]
    
    if len(cluster_data) < 5:  # Skip if too few trades
        return pd.DataFrame()
        
    # Get trade outcomes and adjust for direction
    cluster_trades = cluster_data['trade_outcome']
    if trade_direction == 'short':
        cluster_trades = -cluster_trades
    
    # Basic performance metrics
    total_return = cluster_trades.sum()
    num_trades = len(cluster_trades)
    
    # Create performance dictionary
    cluster_performance = {
        "cluster_label": cluster_label,
        "trade_direction": trade_direction,
        "actual_return": total_return,
        "num_trades": num_trades
    }
    return cluster_performance

def cluster_and_evaluate_price_data(price_data_df):
    price_point_columns = [f"price_point_{i}" for i in range(NUM_PERCEPTUALLY_IMPORTANT_POINTS)]
    feature_columns = price_point_columns + ["day_of_week", "hour", "minute"]
    price_features = price_data_df[feature_columns].values
    clustering_model = clustering_estimator_dict[CLUSTERING_ALGORITHM]
    clustering_model.fit(price_features)
    price_data_df["cluster_label"] = clustering_model.predict(price_features)

    cluster_metrics = []
    for cluster in price_data_df['cluster_label'].unique():
        cluster_data = price_data_df[price_data_df['cluster_label'] == cluster]
        
        if len(cluster_data) < 5:  # Skip clusters with too few trades
            continue
            
        # Determine if cluster is long or short based on mean return
        cluster_mean = cluster_data['trade_outcome'].mean()
        is_long_cluster = cluster_mean >= 0
        
        # Adjust trade outcomes for short trades
        cluster_trades = cluster_data['trade_outcome']
        if not is_long_cluster:
            cluster_trades = -cluster_trades  # Invert returns for short trades
            
        # Basic metrics
        wins = cluster_trades > 0
        losses = cluster_trades < 0
        
        # 1. Win Rate
        win_rate = np.mean(wins) if len(cluster_trades) > 0 else 0
        
        # 2. Risk-adjusted return
        returns_mean = cluster_trades.mean()
        returns_std = cluster_trades.std()
        sharpe = returns_mean / returns_std if returns_std != 0 else 0
        
        # 3. Maximum Drawdown
        cumulative = cluster_trades.cumsum()
        running_max = cumulative.expanding().max()
        drawdowns = cumulative - running_max
        max_drawdown = abs(drawdowns.min()) if len(drawdowns) > 0 else float('inf')
        
        # 4. Profit Factor
        gross_profits = cluster_trades[wins].sum() if any(wins) else 0
        gross_losses = abs(cluster_trades[losses].sum()) if any(losses) else 0
        profit_factor = gross_profits / gross_losses if gross_losses != 0 else float('inf')
        
        # 5. Win/Loss Ratio
        avg_win = cluster_trades[wins].mean() if any(wins) else 0
        avg_loss = abs(cluster_trades[losses].mean()) if any(losses) else float('inf')
        win_loss_ratio = avg_win / avg_loss if avg_loss != 0 else float('inf')
        
        # 6. Consistency Score (lower is better)
        returns_volatility = cluster_trades.std()
        
        cluster_metrics.append({
            'cluster_label': cluster,
            'trade_direction': 'long' if is_long_cluster else 'short',
            'n_trades': len(cluster_trades),
            'win_rate': win_rate,
            'sharpe': sharpe,
            'max_drawdown': max_drawdown,
            'profit_factor': profit_factor,
            'win_loss_ratio': win_loss_ratio,
            'returns_volatility': returns_volatility,
            'avg_return': returns_mean,
            'total_return': cluster_trades.sum(),
            'original_mean_return': cluster_mean
        })
    metrics_df = pd.DataFrame(cluster_metrics)
    
    # Return empty DataFrame if no valid clusters
    if len(metrics_df) == 0:
        return pd.DataFrame(), clustering_model  
        
    # Calculate composite score with direction-aware metrics
    metrics_df['consistency_score'] = (
        metrics_df['win_rate'] * 0.25 +
        metrics_df['sharpe'].clip(0, 3) / 3 * 0.2 +
        (1 / (1 + metrics_df['max_drawdown'])) * 0.2 +
        (metrics_df['profit_factor'].clip(0, 5) / 5) * 0.15 +
        (metrics_df['win_loss_ratio'].clip(0, 3) / 3) * 0.1 +
        (metrics_df['avg_win_streak'] / 5) * 0.1
    )

    # Filter for good clusters regardless of direction
    good_clusters = metrics_df[
        (metrics_df['returns_volatility'] < metrics_df['returns_volatility'].median() * 1.5) &
        (metrics_df['max_drawdown'] < metrics_df['max_drawdown'].median() * 1.2) &
        (metrics_df['win_rate'] > 0.5) &
        (metrics_df['n_trades'] >= 10)
    ]
    
    # Return empty DataFrame if no good clusters
    if len(good_clusters) == 0:
        return pd.DataFrame(), clustering_model  
    
    # Get single best cluster by consistency score
    best_cluster_df = (
        good_clusters
        .sort_values('consistency_score', ascending=False)
        .head(1)
        .reset_index(drop=True)
    )
    return best_cluster_df, clustering_model


# PROJECT_DIR = "/projects/genomic-ml/da2343/ml_project_2"
PROJECT_DIR = "/Users/newuser/Projects/robust_algo_trader"
# Load the config file
config_path = f"{PROJECT_DIR}/settings/config_gfd.json"
with open(config_path) as f:
    config = json.load(f)

instrument_dict = config["traded_instruments"][INSTRUMENT.split("_M15")[0]]
time_scaler = joblib.load(f"{PROJECT_DIR}/unsupervised/kmeans/ts_scaler_2018.joblib")
price_data = pd.read_csv(
    f"{PROJECT_DIR}/data/gen_oanda_data/{INSTRUMENT}_raw_data.csv",
    parse_dates=["time"],
    index_col="time",
)

# Filter date range and apply time scaling
price_data = price_data.loc["2019-01-01":"2024-06-01"]
price_data["year"] = price_data.index.year
price_data["month"] = price_data.index.month
price_data["day_of_week"] = price_data.index.dayofweek
price_data["hour"] = price_data.index.hour
price_data["minute"] = price_data.index.minute
# Calculate ATR as abs(high - low)
price_data["atr"] = (price_data["high"] - price_data["low"]).abs()
# Clip the ATR values
price_data["atr_clipped"] = np.clip(price_data["atr"], instrument_dict['atr_min'], instrument_dict['atr_max'])
# Round time columns after scaling
time_columns = ["day_of_week", "hour", "minute"]
price_data[time_columns] = time_scaler.transform(price_data[time_columns])
# Round price columns
columns_to_round = ['open', 'high', 'low', 'close', 'atr', 'atr_clipped', "day_of_week", "hour", "minute"]
price_data[columns_to_round] = price_data[columns_to_round].round(6)


# print the head of the price data
print("price_data.head()")
print(price_data.head())

# Initialize the sliding window splitter for backtesting
window_splitter = OrderedSlidingWindowSplitter(
    train_weeks=TRAIN_PERIOD, test_weeks=TEST_PERIOD, step_size=1
)

backtest_results = []
for window, (train_indices, test_indices) in enumerate(window_splitter.split(price_data), 1):
    print(f"Processing window {window}...")
    train_data = price_data.iloc[train_indices, :]
    test_data = price_data.iloc[test_indices, :]

    # Prepare training data and perform clustering
    print("Preparing training data and clustering...")
    train_price_data = prepare_data(train_data)
    train_best_clusters, clustering_model = cluster_and_evaluate_price_data(train_price_data)
    if train_best_clusters.empty:
        continue

    # Prepare test data and evaluate cluster performance
    print("Preparing test data and evaluating cluster performance...")
    test_price_data = prepare_data(test_data)
    test_cluster_performance = evaluate_cluster_performance_df(test_price_data, train_best_clusters, clustering_model)
    if test_cluster_performance.empty:
        continue

    # Compile results for this window
    print("Compiling results...")
    window_result = {
        "window": window,
        "train_total_actual_return": train_best_clusters["actual_return"].sum(),
        "train_total_trades": train_best_clusters["num_trades"].sum(),
        
        "test_total_actual_return": test_cluster_performance["actual_return"].sum(),
        "test_total_trades": test_cluster_performance["num_trades"].sum()
    }
    backtest_results.append(window_result)

# Compile final results
results_df = pd.DataFrame(backtest_results)
results_df["train_cumulative_actual_return"] = results_df["train_total_actual_return"].cumsum()
results_df["train_sharpe_ratio"] = calculate_sharpe_ratio(results_df["train_total_annualized_return"].values)

results_df["test_cumulative_actual_return"] = results_df["test_total_actual_return"].cumsum()
results_df["test_sharpe_ratio"] = calculate_sharpe_ratio(results_df["test_total_annualized_return"].values)
results_df["test_inverse_sharpe_ratio"] = calculate_sharpe_ratio( -1 * results_df["test_total_annualized_return"].values)

# Add constant parameters to the results
results_df["max_cluster_labels"] = MAX_CLUSTER_LABELS
results_df["num_clusters"] = NUM_CLUSTERS
results_df["clustering_algorithm"] = CLUSTERING_ALGORITHM
results_df["train_period"] = TRAIN_PERIOD
results_df["test_period"] = TEST_PERIOD
results_df["random_seed"] = RANDOM_SEED
results_df["instrument"] = INSTRUMENT
results_df["num_perceptually_important_points"] = NUM_PERCEPTUALLY_IMPORTANT_POINTS

# save results to csv
out_file = f"results/{param_row}.csv"
results_df.to_csv(out_file, encoding="utf-8", index=False)
print("Backtesting completed.")

In [ ]:
import sys
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import *
from sklearn.mixture import GaussianMixture
from sktime.forecasting.model_selection import SlidingWindowSplitter
from numba import jit
import joblib
import os
import shutil
import json
from sktime.forecasting.model_selection import SlidingWindowSplitter


INSTRUMENT = "EUR_USD_M15"
PROJECT_DIR = "/projects/genomic-ml/da2343/ml_project_2"
# PROJECT_DIR = "/Users/newuser/Projects/robust_algo_trader"
# Load the config file
config_path = f"{PROJECT_DIR}/settings/config.json"
with open(config_path) as f:
    config = json.load(f)

instrument_dict = config["traded_instruments"][INSTRUMENT.split("_M15")[0]]
time_scaler = joblib.load(f"{PROJECT_DIR}/unsupervised/kmeans/ts_scaler_2018.joblib")
price_data = pd.read_csv(
    f"{PROJECT_DIR}/data/gen_oanda_data/{INSTRUMENT}_raw_data.csv",
    parse_dates=["time"],
    index_col="time",
)

# Filter date range and apply time scaling
price_data = price_data.loc["2019-01-01":"2020-01-01"]
price_data['log_close'] = np.log(price_data['close'])
price_data['log_close_diff'] = price_data['log_close'].diff().fillna(0)
price_data["year"] = price_data.index.year
price_data["month"] = price_data.index.month
price_data["day_of_week"] = price_data.index.dayofweek
price_data["hour"] = price_data.index.hour
price_data["minute"] = price_data.index.minute
time_columns = ["day_of_week", "hour", "minute"]
price_data[time_columns] = time_scaler.transform(price_data[time_columns])
# Round price columns
columns_to_round = ['open', 'high', 'low', 'close', 'log_close', "day_of_week", "hour", "minute"]
price_data[columns_to_round] = price_data[columns_to_round].round(6)


In [ ]:
import pandas as pd
import numpy as np

def evaluate_cluster_performance_df(price_data_df, train_best_clusters_df, clustering_model):
    # Prepare features for prediction
    price_point_columns = [f"price_point_{i}" for i in range(NUM_PERCEPTUALLY_IMPORTANT_POINTS)]
    feature_columns = price_point_columns + ["day_of_week", "hour", "minute"]
    
    # Predict clusters for test data
    price_features = price_data_df[feature_columns].values
    price_data_df["cluster_label"] = clustering_model.predict(price_features)
    
    # Get the best cluster label and its direction from training
    best_cluster = train_best_clusters_df.iloc[0]  # Since we only have one best cluster
    cluster_label = best_cluster['cluster_label']
    trade_direction = best_cluster['trade_direction']
    
    # Get trades for the best cluster
    cluster_data = price_data_df[price_data_df['cluster_label'] == cluster_label]
    
    if len(cluster_data) < 5:  # Skip if too few trades
        return pd.DataFrame()
        
    # Get trade outcomes and adjust for direction
    cluster_trades = cluster_data['trade_outcome']
    if trade_direction == 'short':
        cluster_trades = -cluster_trades
    
    # Basic performance metrics
    total_return = cluster_trades.sum()
    num_trades = len(cluster_trades)
    
    # Create performance dictionary
    cluster_performance = {
        "cluster_label": cluster_label,
        "trade_direction": trade_direction,
        "actual_return": total_return,
        "num_trades": num_trades
    }
    return cluster_performance


def cluster_and_evaluate_price_data(price_data_df):
    price_point_columns = [f"price_point_{i}" for i in range(NUM_PERCEPTUALLY_IMPORTANT_POINTS)]
    feature_columns = price_point_columns + ["day_of_week", "hour", "minute"]
    price_features = price_data_df[feature_columns].values
    clustering_model = clustering_estimator_dict[CLUSTERING_ALGORITHM]
    clustering_model.fit(price_features)
    price_data_df["cluster_label"] = clustering_model.predict(price_features)

    cluster_metrics = []
    for cluster in price_data_df['cluster_label'].unique():
        cluster_data = price_data_df[price_data_df['cluster_label'] == cluster]
        
        if len(cluster_data) < 5:  # Skip clusters with too few trades
            continue
            
        # Determine if cluster is long or short based on mean return
        cluster_mean = cluster_data['trade_outcome'].mean()
        is_long_cluster = cluster_mean >= 0
        
        # Adjust trade outcomes for short trades
        cluster_trades = cluster_data['trade_outcome']
        if not is_long_cluster:
            cluster_trades = -cluster_trades  # Invert returns for short trades
            
        # Basic metrics
        wins = cluster_trades > 0
        losses = cluster_trades < 0
        
        # 1. Win Rate
        win_rate = np.mean(wins) if len(cluster_trades) > 0 else 0
        
        # 2. Risk-adjusted return
        returns_mean = cluster_trades.mean()
        returns_std = cluster_trades.std()
        sharpe = returns_mean / returns_std if returns_std != 0 else 0
        
        # 3. Maximum Drawdown
        cumulative = cluster_trades.cumsum()
        running_max = cumulative.expanding().max()
        drawdowns = cumulative - running_max
        max_drawdown = abs(drawdowns.min()) if len(drawdowns) > 0 else float('inf')
        
        # 4. Profit Factor
        gross_profits = cluster_trades[wins].sum() if any(wins) else 0
        gross_losses = abs(cluster_trades[losses].sum()) if any(losses) else 0
        profit_factor = gross_profits / gross_losses if gross_losses != 0 else float('inf')
        
        # 5. Win/Loss Ratio
        avg_win = cluster_trades[wins].mean() if any(wins) else 0
        avg_loss = abs(cluster_trades[losses].mean()) if any(losses) else float('inf')
        win_loss_ratio = avg_win / avg_loss if avg_loss != 0 else float('inf')
        
        # 6. Consistency Score (lower is better)
        returns_volatility = cluster_trades.std()
        
        cluster_metrics.append({
            'cluster_label': cluster,
            'trade_direction': 'long' if is_long_cluster else 'short',
            'n_trades': len(cluster_trades),
            'win_rate': win_rate,
            'sharpe': sharpe,
            'max_drawdown': max_drawdown,
            'profit_factor': profit_factor,
            'win_loss_ratio': win_loss_ratio,
            'returns_volatility': returns_volatility,
            'avg_return': returns_mean,
            'total_return': cluster_trades.sum(),
            'original_mean_return': cluster_mean
        })
    metrics_df = pd.DataFrame(cluster_metrics)
    
    # Return empty DataFrame if no valid clusters
    if len(metrics_df) == 0:
        return pd.DataFrame(), clustering_model  
        
    # Calculate composite score with direction-aware metrics
    metrics_df['consistency_score'] = (
        metrics_df['win_rate'] * 0.25 +
        metrics_df['sharpe'].clip(0, 3) / 3 * 0.2 +
        (1 / (1 + metrics_df['max_drawdown'])) * 0.2 +
        (metrics_df['profit_factor'].clip(0, 5) / 5) * 0.15 +
        (metrics_df['win_loss_ratio'].clip(0, 3) / 3) * 0.1 +
        (metrics_df['avg_win_streak'] / 5) * 0.1
    )

    # Filter for good clusters regardless of direction
    good_clusters = metrics_df[
        (metrics_df['returns_volatility'] < metrics_df['returns_volatility'].median() * 1.5) &
        (metrics_df['max_drawdown'] < metrics_df['max_drawdown'].median() * 1.2) &
        (metrics_df['win_rate'] > 0.5) &
        (metrics_df['n_trades'] >= 10)
    ]
    
    # Return empty DataFrame if no good clusters
    if len(good_clusters) == 0:
        return pd.DataFrame(), clustering_model  
    
    # Get single best cluster by consistency score
    best_cluster_df = (
        good_clusters
        .sort_values('consistency_score', ascending=False)
        .head(1)
        .reset_index(drop=True)
    )
    return best_cluster_df, clustering_model

In [ ]:
# 1. For each window
def compile_window_results(window, train_best_clusters, test_cluster_performance):
    window_result = {
        "window": window,
        # Training metrics (single best cluster)
        "train_actual_return": train_best_clusters["actual_return"].iloc[0],  # No sum needed, single value
        "train_num_trades": train_best_clusters["num_trades"].iloc[0],    # No sum needed, single value
        "train_direction": train_best_clusters["trade_direction"].iloc[0],
        
        # Test metrics (single cluster performance)
        "test_actual_return": test_cluster_performance["actual_return"].iloc[0],  # No sum needed, single value
        "test_num_trades": test_cluster_performance["num_trades"].iloc[0],    # No sum needed, single value
        "test_direction": test_cluster_performance["trade_direction"].iloc[0]
    }
    return window_result

# 2. Calculate final metrics
def calculate_backtest_metrics(results_df):
    """
    Calculate overall backtest performance metrics
    """
    # Calculate returns for Sharpe Ratio
    test_returns = results_df['test_actual_return']
    
    # Sharpe Ratio calculation
    # Assuming returns are already in excess of risk-free rate
    sharpe_ratio = np.mean(test_returns) / np.std(test_returns) if np.std(test_returns) != 0 else 0
    
    # Annualize Sharpe (assuming daily returns)
    annual_factor = np.sqrt(252)  # Trading days in a year
    annual_sharpe = sharpe_ratio * annual_factor
    
    # Calculate cumulative return
    cumulative_return = (1 + test_returns).prod() - 1  # Compound returns
    
    metrics = {
        "total_windows": len(results_df),
        "total_test_trades": results_df['test_num_trades'].sum(),
        "average_trades_per_window": results_df['test_num_trades'].mean(),
        
        "cumulative_return": cumulative_return,
        "average_return_per_window": test_returns.mean(),
        "return_std": test_returns.std(),
        "sharpe_ratio": annual_sharpe,
        
        "profitable_windows": (test_returns > 0).mean(),
        "long_windows": (results_df['test_direction'] == 'long').mean(),
        "short_windows": (results_df['test_direction'] == 'short').mean()
    }
    
    return metrics

# Example usage:
# 1. During backtesting
backtest_results = []
for window in windows:
    # ... your existing backtesting code ...
    
    window_result = compile_window_results(
        window=window,
        train_best_clusters=train_best_clusters,
        test_cluster_performance=test_cluster_performance
    )
    backtest_results.append(window_result)

# 2. After backtesting
results_df = pd.DataFrame(backtest_results)
final_metrics = calculate_backtest_metrics(results_df)

print("\nBacktest Results:")
print(f"Total Windows: {final_metrics['total_windows']}")
print(f"Total Test Trades: {final_metrics['total_test_trades']}")
print(f"Cumulative Return: {final_metrics['cumulative_return']:.4f}")
print(f"Annualized Sharpe Ratio: {final_metrics['sharpe_ratio']:.4f}")
print(f"Profitable Windows: {final_metrics['profitable_windows']:.2%}")


Backtest Performance Summary

Overall Statistics:
Total Windows: 10
Total Trades: 53
Avg Trades per Window: 5.30

Performance Metrics:
Cumulative Return: 1.4262%
Average Return: 0.1418%
Return Std Dev: 0.1263%
Sharpe Ratio: 16.8170
Maximum Drawdown: 0.0031%

Strategy Characteristics:
Win Ratio: 90.00%
Long Ratio: 70.00%

Warnings:
